In [ ]:
!pip install accelerate bitsandbytes

In [ ]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

In [15]:
import pandas as pd

# Assuming 'writer_of_questions.csv' is in your current directory or you provide the correct path.
df = pd.read_csv('writer_of_episodes_questions.csv')

In [19]:
prompts = []
for index, row in df.iterrows():
    question = row['Question']
    messages = [
        {"role": "system", "content": "You are a Seinfeld expert, skilled in generating scripts in the iconic style of the show. You understand the characters' distinct personalities, and comedic timing. In addition to writing scripts, you are a trivia master, able to answer detailed questions about the show, such as who said specific lines and who wrote particular episodes"},
        {"role": "system", "content": "when asked who is the writer of an episode, answer in the following way if there is only one writer: The writer of [name of episode] is [name of writer]"},
        {"role": "system", "content": "when asked who is the writer of an episode, answer in the following way if there are more than one writer: The writers of [name of episode] are [name of writers]"},
        {"role": "user", "content": question},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    prompts += [prompt]

In [ ]:


terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompts,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
)

for index,output in enumerate(outputs):
    print([output["generated_text"][len(prompts[index]):]])

#questions += [question]
#answers += [row['Answer']]
#model_answers += [outputs[0]["generated_text"][len(prompt):]]

#new_df = pd.DataFrame({'Question': questions, 'Answer': answers, 'ModelAnswer': model_answers})
#new_df.to_csv('writer_of_episodes_questions_model_answers.csv', mode='w', header=False, index=False)